# PyTorch and TensorFlow

我之前一直没理解为什么 PyTorch 的一些 API 输入一批数据的时候要规定数据的维度的第二维是 batch size，而不是放在第一个维度。

例如 NLP 里的一些 module 的输入维度是 (max_length, batch_size, input_size)，即第一维是每条语句的最大长度（词数量），第二维是批大小，第三位是词嵌入的维度大小。

后来看了 PyTorch Chatbot 的教程才知道，这样做的好处是，对输入数据取第一个维度的下标的时候，一次就能取到所有 batch 的该下标对应的 time-step。这在 GPU 上并行的时候就会方便很多。

如果把 batch size 放在第一个维度，那么如果想取所有 batch 的某一个 time-step，就要用到 Python 的 slice 操作，例如 

In [11]:
import numpy as np
data = np.array([[11, 12, 13],
                 [21, 22, 23],
                 [31, 32, 33]])
print(data[:,1])

[12 22 32]
